# Multiple Scatter Simulations with `k100Sim`

The previous `iPy` notebook `ms_simulation_explore.ipynb` explored ways that I can use `pandas` to reduce the simulated dataset. Here I want to do this for the entire dataset. To start off with I'll try this in a notebook, and use the standard format for the data reduction into `h5` files produced by:

```./sima2py.py --regex 'Cf252_k100_0x0000_10M_\S+.txt' --filedir /data/chocula/villaa/k100Sim_Data/252Cf/ --outfile /data/chocula/villaa/k100Sim_Data/252Cf/Cf252_0x0000.h5```

The hope is then that the analysis here can be absporbed into `sima2py.py` and be a better way to select and store this simulation data, without going through ROOT or anything. 

In [1]:
import numpy as np
import pandas as pd
import sima2py as sapy

In [2]:
#play around with some hits data stored in h5 file
#===============to suppress h5py warning see:
#https://github.com/h5py/h5py/issues/961
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()
f = h5py.File("/data/chocula/villaa/k100Sim_Data/252Cf/Cf252_0x0000.h5","r")

for i in f:
    print(i)

hits = f['geant4/hits']

geant4


In [ ]:
#using the examples above let's get byzantine and make a bad-ass data frame.

data=hits
#first do some cuts:
#first some hit-level cuts
cHVDet = np.zeros(np.shape(data)[0],dtype=bool)
cZeroEdep = np.zeros(np.shape(data)[0],dtype=bool)
cNeutron = np.zeros(np.shape(data)[0],dtype=bool)
cGamma = np.zeros(np.shape(data)[0],dtype=bool)
cNR = np.zeros(np.shape(data)[0],dtype=bool)

cHVDet[data[:,1]==1] = True
cZeroEdep[data[:,6]==0] = True
cNeutron[data[:,4]==2112] = True
cGamma[data[:,4]==22] = True
cNR[data[:,4]>3000] = True


#now some event-level cuts
#evWithHVhits = newev[cHVDet & ~cZeroEdep]
#cWithHVHits = np.isin(newev,evWithHVhits)
#print(np.sum(cWithHVHits))
#print(np.shape(np.unique(evWithHVhits)))


#now make a dataframe with the restricted data
nr_data = data[:,[0,4,5,6,21]]
nr_data = nr_data[cHVDet&~cZeroEdep&cNR,:]
print(np.shape(nr_data))
nr_dataframe = pd.DataFrame(data=nr_data)

#print(nr_dataframe)

#print(np.max(nr_dataframe.groupby([0,1],axis=0).size()))
max_vec = np.max(nr_dataframe.groupby([0,1],axis=0).size())

evec = np.zeros((0,max_vec))
nhit = np.zeros((0,1))

for i in nr_dataframe.groupby([0,1],axis=0)[3].apply(list):
    #print(i)
    #print(np.shape(i))
    vector = np.zeros((1,max_vec))
    #print(np.shape(vector[0,0:np.shape(i)[0]]))
    vector[0,0:np.shape(i)[0]] = np.transpose(np.asarray(i))
    evec = np.append(evec,vector,0)
    nhit = np.append(nhit,np.shape(i)[0])
    
print(np.shape(evec))
print(np.shape(nhit))
#print(max_vec)
#print(np.sum(evec,1))
print(np.sum(nhit[nhit==1]))